In [ ]:
pip --version

pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)


In [ ]:
pip install dicom2nifti

In [ ]:
!pip uninstall -y HD-BET

Found existing installation: HD_BET 2.0.1
Uninstalling HD_BET-2.0.1:
  Successfully uninstalled HD_BET-2.0.1


In [ ]:
!git clone https://github.com/MIC-DKFZ/HD-BET.git
%cd HD-BET
!pip install -e .


Cloning into 'HD-BET'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (69/69), done.
Receiving objects: 100% (313/313), 73.82 KiB | 6.71 MiB/s, done.
remote: Total 313 (delta 104), reused 125 (delta 92), pack-reused 152 (from 1)
Resolving deltas: 100% (183/183), done.
/content/HD-BET/HD-BET
Obtaining file:///content/HD-BET/HD-BET
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Building editable for HD_BET (pyproject.toml) ... done
  Created wheel for HD_BET: filename=hd_bet-2.0.1-0.editable-py3-none-any.whl size=7348 sha256=483a43f063ee3f1872492b0f87658d65622131f25406acbad57f6dfe1b4a921c
  Stored in directory: /tmp/pip-ephem-wh

In [ ]:
!pip show HD-BET

Name: HD_BET
Version: 2.0.1
Summary: Tool for brain extraction
Home-page: https://github.com/MIC-DKFZ/hd-bet
Author: 
Author-email: Fabian Isensee <f.isensee@dkfz.de>
License: Apache-2.0
Location: /usr/local/lib/python3.12/dist-packages
Editable project location: /content/HD-BET/HD-BET
Requires: nnunetv2, numpy, scikit-image, SimpleITK, torch
Required-by: 


In [ ]:
!mkdir -p /content/input
!mkdir -p /content/hd_bet_masks
!mkdir -p /content/hd_bet_brains

In [ ]:
import os
import subprocess
import nibabel as nib
import numpy as np


def safe_output_name(fname, suffix):
    # 확장자 제거 (.nii / .nii.gz 모두 대응)
    if fname.endswith(".nii.gz"):
        base = fname[:-7]   # ".nii.gz" 제거
    elif fname.endswith(".nii"):
        base = fname[:-4]   # ".nii" 제거
    else:
        base = fname
    return base + suffix + ".nii.gz"

def run_hd_bet_cli(input_path: str, output_mask_path: str, device: str = "cuda", disable_tta: bool = False):
    cmd = [
        "hd-bet",
        "-i", input_path,
        "-o", output_mask_path,
        "-device", device,
    ]
    if disable_tta:
        cmd += ["--disable_tta"]
    file_name = os.path.basename(input_path)

    result = subprocess.run(cmd, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("Done")

    if result.returncode != 0:
        raise RuntimeError(f"hd-bet failed (exit code {result.returncode})")


def apply_mask_to_image(input_path: str, mask_path: str, out_path: str):
    """
    마스크를 이미지에 적용해서 뇌 영역만 남긴 이미지 저장
    """
    img = nib.load(input_path)
    mask = nib.load(mask_path)
    img_data = img.get_fdata()
    mask_data = mask.get_fdata()

    # 마스크가 binary (0 또는 1)라는 가정하에, 이미지 곱셈
    new_data = img_data * (mask_data > 0)

    out_img = nib.Nifti1Image(new_data, img.affine, img.header)
    nib.save(out_img, out_path)


def batch_process(input_dir: str, output_mask_dir: str, output_brain_dir: str):
    os.makedirs(output_mask_dir, exist_ok=True)
    os.makedirs(output_brain_dir, exist_ok=True)
    i = 1
    for fname in os.listdir(input_dir):
        if not fname.endswith(".nii") and not fname.endswith(".nii.gz"):
            continue

        in_path = os.path.join(input_dir, fname)
        mask_out = os.path.join(output_mask_dir, safe_output_name(fname, "_mask"))
        brain_out = os.path.join(output_brain_dir, safe_output_name(fname, "_brain"))


        print(f"{i}.Processing:{fname}")
        run_hd_bet_cli(in_path, mask_out, device="cuda", disable_tta=False)

        # 마스크를 이용해 뇌영역 이미지를 별도로 생성
        apply_mask_to_image(in_path, mask_out, brain_out)
        i += 1

# 예시 디렉토리
input_dir = "/content/input"
output_mask_dir = "/content/hd_bet_masks"
output_brain_dir = "/content/hd_bet_brains"

batch_process(input_dir, output_mask_dir, output_brain_dir)

In [ ]:
!nvidia-smi

Fri Oct 24 06:04:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from ipywidgets import interact, IntSlider

# --- NIfTI 파일 경로 ---
path = "/content/I817508 (1).nii.gz"

# --- 데이터 로드 ---
img = nib.load(path)
data = img.get_fdata()

# --- 뷰어 함수 정의 ---
def show_slices(sagittal, coronal, axial):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Sagittal (좌↔우)
    axes[0].imshow(np.rot90(data[sagittal, :, :]), cmap="gray")
    axes[0].set_title(f"Sagittal (x={sagittal})")
    axes[0].axis("off")

    # Coronal (앞↔뒤)
    axes[1].imshow(np.rot90(data[:, coronal, :]), cmap="gray")
    axes[1].set_title(f"Coronal (y={coronal})")
    axes[1].axis("off")

    # Axial (위↔아래)
    axes[2].imshow(np.rot90(data[:, :, axial]), cmap="gray")
    axes[2].set_title(f"Axial (z={axial})")
    axes[2].axis("off")

    plt.show()

# --- 슬라이더 설정 ---
interact(
    show_slices,
    sagittal=IntSlider(min=0, max=data.shape[0]-1, step=1, value=data.shape[0]//2),
    coronal=IntSlider(min=0, max=data.shape[1]-1, step=1, value=data.shape[1]//2),
    axial=IntSlider(min=0, max=data.shape[2]-1, step=1, value=data.shape[2]//2)
)


interactive(children=(IntSlider(value=64, description='sagittal', max=127), IntSlider(value=64, description='c…

<function __main__.show_slices(sagittal, coronal, axial)>

In [ ]:
# 저장준비용
import os
import shutil
from google.colab import drive

# Google 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
#파일 개수
num_files = len(os.listdir('/content/drive/MyDrive/hd_bet'))
print(num_files)

In [ ]:
# brain저장용
# 원본 경로
source_path = '/content/hd_bet_brains'

# 목적지 경로
destination_path = '/content/drive/MyDrive/hd_bet'

# 목적지 경로가 없으면 생성
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# '/content' 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path):
    file_path = os.path.join(source_path, filename)
    dest_file_path = os.path.join(destination_path, filename)

    if os.path.exists(dest_file_path):
        print(f"Skipped (already exists): {filename}")
        continue

    shutil.move(file_path, destination_path)

In [ ]:
# mask저장용
# 원본 경로
source_path = '/content/hd_bet_masks'

# 목적지 경로
destination_path = '/content/drive/MyDrive/mask'

# 목적지 경로가 없으면 생성
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# '/content' 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path):
    file_path = os.path.join(source_path, filename)
    dest_file_path = os.path.join(destination_path, filename)

    if os.path.exists(dest_file_path):
        print(f"Skipped (already exists): {filename}")
        continue

    shutil.move(file_path, destination_path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf /content/input
!rm -rf /content/hd_bet_brains
!rm -rf /content/hd_bet_masks

In [ ]:
import nibabel as nib
import numpy as np
img = nib.load("I817508.nii.gz")
data = img.get_fdata()
print(np.unique(data)[:10])  # 최소~최대값 확인
print(np.count_nonzero(data==0) / data.size)

[ 0.          4.00195331  6.99763513 15.00154176 15.99624692 17.00251842
 17.99722358 19.00349508 23.00544839 24.00015355]
0.9922013716264204
